### INIT

In [ ]:
generate_knowledge.add_repo_root_path()
import openai_setup

OPENAI_API_KEY = openai_setup.conf['key']
OPENAI_PROJECT = openai_setup.conf['project']
OPENAI_ORGANIZATION = openai_setup.conf['organization']
DEFAULT_LLM_MODEL = "gpt-4o-mini"

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY, model="text-embedding-ada-002")
llm = ChatOpenAI(model=DEFAULT_LLM_MODEL, temperature=0.1, openai_api_key=OPENAI_API_KEY, openai_organization = OPENAI_ORGANIZATION)

In [ ]:
# Define file paths for YAML configurations
files = {
    'agents': '../config/agents.yml',
    'tasks': '../config/tasks.yml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

print(agents_config)
print(tasks_config)

In [ ]:
from crewai import Agent, Task, Crew

# Creating Agents
interpretation_agent = Agent(
  config=agents_config['interpretation_agent'],
  llm = llm
)

evaluation_agent = Agent(
  config=agents_config['evaluation_agent'],
  llm = llm
)

# Creating Tasks
interpretation_task = Task(
  config=tasks_config['interpretation_task'],
  agent=interpretation_agent
)

evaluation_task = Task(
  config=tasks_config['evaluation_task'],
  agent=evaluation_agent
)

crew = Crew(
  agents=[
    interpretation_agent,
    evaluation_agent
  ],
  tasks=[
    interpretation_task,
    evaluation_task
  ],
  verbose=True
)

In [ ]:
from crewai import Flow
from crewai.flow.flow import listen, start

class dbtChatFlow(Flow):
    @start()
    async def interpret_prompt(self):
        """
        Step 1: Process the user's prompt using the interpretation agent.
        """
        # Asegúrate de que los inputs estén en el estado
        if "user_prompt" not in self.state:
            raise ValueError("No user_prompt provided in the inputs.")

        user_prompt = self.state["user_prompt"]
        interpretation_result = crew.agents[0].kickoff(user_prompt)
        self.state["interpretation_result"] = interpretation_result
        return interpretation_result

    @listen(lambda state: "interpretation_result" in state)
    async def evaluate_interpretation(self):
        """
        Step 2: Use the evaluation agent to process the interpretation result.
        """
        interpretation_result = self.state.get("interpretation_result")
        if not interpretation_result:
            raise ValueError("interpretation_result is missing in the state.")

        evaluation_result = crew.agents[1].kickoff(interpretation_result)
        self.state["evaluation_result"] = evaluation_result
        return evaluation_result

flow = dbtChatFlow()
#flow.plot()

In [ ]:
user_input = "give me all the models related with the orders dbt model"

flow.update_state({"user_prompt": user_input})
result = await flow.kickoff_async()

print("Pipeline Result:", result)
